In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import mne
import keras
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import random
from torch import optim
import argparse
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import torch.nn.functional as F
from os import walk

In [3]:
DEVICE = torch.device('cuda:0')

In [4]:
def pad(rawdata, seglength):
    pad_length = -rawdata.size()[0] % seglength
    padded = np.concatenate((rawdata, torch.zeros((pad_length, rawdata.size()[1]))), 0)
    return padded

In [5]:
def to_epochs(rawdata, seg_length):
    data = pad(rawdata, seg_length)
    """
    split = torch.Tensor([])
    for element in torch.split(data, seg_length):
        split = torch.cat((split, element))
        """
    split = np.array(np.split(data, seg_length))
    return split.reshape(-1, seg_length, 20)

In [6]:
def to_gpu(x, on_cpu=False, gpu_id=None):

    if torch.cuda.is_available() and not on_cpu:
        x = x.cuda(gpu_id)
    return x

In [7]:
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(d, device) for d in data]
    return data.to(device, non_blocking=True)

In [14]:
def Data(path, train_ratio, seglength, class_num):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    patients = []
    for i in range(len(names)):
        patients.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))

    patients_epochs = to_epochs(patients[0], seglength)
    for patient in patients:
        patients_epochs = np.concatenate((patients_epochs, to_epochs(patient, seglength)), axis = 0)
    
    np.random.shuffle(patients_epochs)
    idx = (np.ceil(0.75 * len(patients_epochs))).astype(int)
    train = torch.Tensor(patients_epochs[0:idx])
    test = torch.tensor(patients_epochs[idx:])
    train_labels = torch.Tensor(np.ones(len(train)).reshape(-1) * class_num)
    test_labels = torch.Tensor(np.ones(len(test)).reshape(-1) * class_num)
    

    return train, test, train_labels, test_labels

In [24]:
def PNES(path, train_ratio, seglength):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    pnes = []
    for i in range(len(names)):
        pnes.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))

    pnes_epochs = to_epochs(pnes[0], seglength)
    for patient in pnes:
        pnes_epochs = np.concatenate((pnes_epochs, to_epochs(patient, seglength)), axis = 0)
    
    np.random.shuffle(pnes_epochs)
    idx = (np.ceil(train_ratio * len(pnes_epochs))).astype(int)
    train = torch.Tensor(pnes_epochs[0:idx])
    test = torch.Tensor(pnes_epochs[idx:])
    train_labels = torch.Tensor(np.ones(len(train)).reshape(-1)) * 3
    test_labels = torch.Tensor(np.ones(len(test)).reshape(-1)) * 3

    return train, test, train_labels, test_labels

In [10]:
def FOCAL(path, train_ratio, seglength):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    focal = []
    for i in range(len(names)):
        focal.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))

    focal_epochs = to_epochs(focal[0], seglength)
    for patient in focal:
        focal_epochs = np.concatenate((focal_epochs, to_epochs(patient, seglength)), axis = 0)
    
    np.random.shuffle(focal_epochs)
    idx = (np.ceil(0.75 * len(focal_epochs))).astype(int)
    train = torch.Tensor(focal_epochs[0:idx])
    test = torch.Tensor(focal_epochs[idx:])
    train_labels = torch.Tensor(np.ones(len(train)).reshape(-1))
    test_labels = torch.Tensor(np.ones(len(test)).reshape(-1))
    
    return train, test, train_labels, test_labels

In [11]:
def IGE(path, train_ratio, seglength):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    ige = []
    for i in range(len(names)):
        ige.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))

    ige_epochs = to_epochs(ige[0], seglength)
    for patient in ige:
        ige_epochs = np.concatenate((ige_epochs, to_epochs(patient, seglength)), axis = 0)
    
    np.random.shuffle(ige_epochs)
    idx = (np.ceil(0.75 * len(ige_epochs))).astype(int)
    train = torch.Tensor(ige_epochs[0:idx])
    test = torch.Tensor(ige_epochs[idx:])
    train_labels = torch.Tensor(np.ones(len(train)).reshape(-1) * 2)
    test_labels = torch.Tensor(np.ones(len(test)).reshape(-1) * 2)

    return train, test, train_labels, test_labels

In [15]:
def TLE(path, train_ratio, seglength):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    temporal = []
    for i in range(len(names)):
        temporal.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))

    temporal_epochs = to_epochs(temporal[0], seglength)
    for patient in temporal:
        temporal_epochs = np.concatenate((temporal_epochs, to_epochs(patient, seglength)), axis = 0)
    
    np.random.shuffle(temporal_epochs)
    idx = (np.ceil(0.75 * len(temporal_epochs))).astype(int)
    train = torch.Tensor(temporal_epochs[0:idx])
    test = torch.tensor(temporal_epochs[idx:])
    train_labels = torch.Tensor(np.ones(len(train)).reshape(-1) * 4)
    test_labels = torch.Tensor(np.ones(len(test)).reshape(-1) * 4)
    

    return train, test, train_labels, test_labels

In [25]:
#ige_train, ige_test, ige_train_labels, ige_test_labels = IGE('D:\EEG\IGE', seglength = 9000, train_ratio = 0.75)
#focal_train, focal_test, focal_train_labels, focal_test_labels = FOCAL('D:\EEG\FOCAL', train_ratio = 0.8, seglength = 9000)
#pnes_train, pnes_test, pnes_train_labels, pnes_test_labels = PNES('D:\EEG\PNES', seglength = 9000, train_ratio = 0.75)
#focal_train, focal_test, focal_train_labels, focal_test_labels = Data('D:\EEG\FOCAL', train_ratio = 0.8, seglength = 9000, class_num = 1)
#temporal_train, temporal_test, temporal_train_labels, temporal_test_labels = TLE('D:\EEG\TLE', train_ratio = 0.8, seglength = 9000)
"""
print(f"train shape: {temporal_train.shape}")
print(f"test shape: {temporal_test.shape}")
print(f"train labels shape: {temporal_train_labels.shape}")
print(f"test labels shape: {temporal_test_labels.shape}")
"""

'\nprint(f"train shape: {temporal_train.shape}")\nprint(f"test shape: {temporal_test.shape}")\nprint(f"train labels shape: {temporal_train_labels.shape}")\nprint(f"test labels shape: {temporal_test_labels.shape}")\n'